# 0. Импорт и загрузка данных

## 0.1 Импорт

In [ ]:
import numpy as np
import osmnx as ox
import random

from utils.base_model import BaseModel
from utils.planter import *
from utils.ueqi import count_change_capacity

np.random.seed(0)
random.seed(0)

## 0.2 Функции

In [ ]:
# Получение значений UEQI для города
def get_ueqi(wff, name):
    city = wff.cities[wff.cities['region_city'].str.contains(name)]
    return city[
                ['region_city',
                "ueqi_residential",
                "ueqi_street_networks",
                "ueqi_green_spaces",
                "ueqi_public_and_business_infrastructure",
                "ueqi_social_and_leisure_infrastructure",
                "ueqi_citywide_space"]
            ].round(3)

## 0.3 Загрузка и подготовка данных

### 0.3.1 Данные из и для TownsNet

In [11]:
region = BaseModel()
region.from_pickle('data/lo_region.pickle') # Модель из TownsNet
supporting_cities = gpd.read_parquet("data/1_polygons.parquet") # Опорные города ЛО
anchor_settlement = pd.read_csv('data/anchor_settlement.csv') # Города/деревни ЛО
ueqi_df = region.calculate_ueqi() # Расчет UEQI по новой формуле

### 0.3.2 Первичное обновление UEQI для Planter
потому что мы не знаем как подсчитаны старые UEQI для городов

In [12]:
# Загрузка модели
wff = WorkForceFlows.from_pickle('data/wff_1812 new.pkl')

Class instance loaded from data/wff_1812 new.pkl


In [13]:
ueqi_df[ueqi_df.town_name == 'Бокситогорск']

,town_name,population,geometry,capacity_cafe__coffee,capacity_kindergarten,capacity_school,capacity_public_transport_stop,capacity_library,capacity_parking,capacity_railway_station,...,count_car_service,count_fire_station,settlement_name,district_name,ueqi_residential,ueqi_street_networks,ueqi_green_spaces,ueqi_public_and_business_infrastructure,ueqi_social_and_leisure_infrastructure,ueqi_citywide_space
id,,,,,,,,,,,,,,,,,,,,,
121,Бокситогорск,8001,POINT (548021.065 6593118.772),250.0,1500.0,1000.0,1500.0,500.0,2500.0,1250.0,...,0,0,Бокситогорское городское поселение,Бокситогорский муниципальный район,49.994,81.24,46.869,100.0,49.994,9.374


In [14]:
name = "Ленинградская область, Бокситогорск" # Можно узнать все города из wff.cities

# Можно взять параметры из ueqi_df[ueqi_df.town_name == 'Мурино'] ← ВАЖНО! Убедитесь, что вы используете правильное название города без области, но для wff с областью
new_params = { # Первичное обновление UEQI для Planter
    "ueqi_residential":                         49.994,
    "ueqi_street_networks":                     81.24,
    "ueqi_green_spaces":                        46.869,
    "ueqi_public_and_business_infrastructure":  100.0,
    "ueqi_social_and_leisure_infrastructure":   49.994,
    "ueqi_citywide_space":                      9.374,
}

wff.update_city_params(name, new_params)
wff.recalculate_after_update()

Updated parameters for Ленинградская область, Бокситогорск
Recalculating after updating parameters


100%|██████████| 1106/1106 [00:01<00:00, 769.57it/s]


Recalculation complete.


In [15]:
# Выделение городов только по ЛО и сохранение новых значений постоянного населения
area = ox.geocode_to_gdf("Ленинградская область")
lo_cities = wff.cities.clip(area.to_crs(3857)).copy()
lo_cities['population'] = lo_cities['population'] + lo_cities['flows_in'] - lo_cities['flows_out']
lo_cities['region_city'] = lo_cities['region_city'].str.split(', ', expand=True)[1] # У TownsNet названия городов без областей, поэтому нужно удалить их из названия
lo_cities[['region_city','population']].to_csv("data/population.csv", index=False)

### 0.2.3 Обновление населения

In [16]:
# update_population(region.model, towns_df, path='data/population.csv')

population = pd.read_csv('data/population.csv') 
region.update_population(population)

ueqi_df = region.calculate_ueqi()
region[121] # Проверим Бокситогорск

### ====== Принимаем это за базовое население ====== ###

town_name                                               Бокситогорск
population                                                   15849.0
geometry                 POINT (548021.0652682963 6593118.771699116)
capacity_cafe__coffee                                          250.0
capacity_kindergarten                                         1500.0
                                            ...                     
count_railway_station                                              0
count_car_service                                                  0
count_fire_station                                                 0
settlement_name                   Бокситогорское городское поселение
district_name                     Бокситогорский муниципальный район
Name: 121, Length: 95, dtype: object

# 1. Выбор города

In [17]:
name = 'Бокситогорск'
get_ueqi(wff, name) # Просмотр нынешных значений UEQI для города

,region_city,ueqi_residential,ueqi_street_networks,ueqi_green_spaces,ueqi_public_and_business_infrastructure,ueqi_social_and_leisure_infrastructure,ueqi_citywide_space
106,"Ленинградская область, Бокситогорск",49.994,81.24,46.869,100.0,49.994,9.374


# 2. Изменение UEQI

In [18]:
new_params = {
    # "ueqi_residential":                         49.994,
    # "ueqi_street_networks":                     81.24,
    "ueqi_green_spaces":                        56.869, # +10 UEQI
    # "ueqi_public_and_business_infrastructure":  100.0,
    # "ueqi_social_and_leisure_infrastructure":   49.994,
    # "ueqi_citywide_space":                      9.374,
}

# Update the city data in the DataFrame
wff.update_city_params('Ленинградская область, Бокситогорск', new_params)
wff.recalculate_after_update()

# Выделение области
area = ox.geocode_to_gdf("Ленинградская область")
highlighted_cities = wff.cities.clip(area.to_crs(3857)).copy()
highlighted_cities['population'] = highlighted_cities['population'] + highlighted_cities['flows_in'] - highlighted_cities['flows_out']
highlighted_cities['region_city'] = highlighted_cities['region_city'].str.split(', ', expand=True)[1] # У TownsNet названия городов без областей, поэтому нужно удалить их из названия
highlighted_cities[['region_city', 'population']].to_csv("data/population_new.csv", index=False)

Updated parameters for Ленинградская область, Бокситогорск
Recalculating after updating parameters


100%|██████████| 1106/1106 [00:01<00:00, 766.02it/s]


Recalculation complete.


In [19]:
# Обновим население городов после изменения UEQI
population_new = pd.read_csv('data/population_new.csv') 
region.update_population(population_new)
region[121] # Проверим Бокситогорск

### ====== Принимаем это за новое население ====== ###

town_name                                               Бокситогорск
population                                                   20200.0
geometry                 POINT (548021.0652682963 6593118.771699116)
capacity_cafe__coffee                                          250.0
capacity_kindergarten                                         1500.0
                                            ...                     
count_railway_station                                              0
count_car_service                                                  0
count_fire_station                                                 0
settlement_name                   Бокситогорское городское поселение
district_name                     Бокситогорский муниципальный район
Name: 121, Length: 95, dtype: object

# 3. Расчет матриц переходов по новым UEQI

## 3.1 Расчет количества и добавление необходимого сервиса

In [20]:
# Посмотим, на сколько нужно поднять
change_capacity = count_change_capacity(region.towns, name, 10)
print(f"Изменение capacity на 10% для города {name}: {change_capacity} мест")

Изменение capacity на 10% для города Бокситогорск: 2020 мест


In [21]:
updated_service = region.update_service(name, change_capacity, new_params)
print(f'Случайный обновленный сервис: {updated_service}')

print('======= Таблица для проверки новых сервисов =======')
region.services[(region.services.town.str.contains(name)) & (region.services.service_type == updated_service)]

Случайный обновленный сервис: park
======= Таблица для проверки новых сервисов =======


,town,service_type,geometry,capacity
175,Бокситогорск,park,POINT (547977.638 6593091.073),250
176,Бокситогорск,park,POINT (548120.962 6592967.185),250
177,Бокситогорск,park,POINT (548140.672 6592774.843),250
178,Бокситогорск,park,POINT (548041.259 6593578.502),250
179,Бокситогорск,park,POINT (548116.559 6593224.947),250
180,Бокситогорск,park,POINT (547652.597 6593571.952),250
181,Бокситогорск,park,POINT (547814.714 6593576.322),250
182,Бокситогорск,park,POINT (547745.356 6593398.051),2020


## 3.2 Расчет матриц переходов

In [ ]:
region.calculate_provision(['park'])

# 4. Расчет данных для городов

## 4.1 Чтение матриц переходов

In [ ]:
use_updated = False

if use_updated:
    population = pd.read_csv('data/population_new.csv') 
    region.update_population(population)
else:
    population = pd.read_csv('data/population.csv') 
    region.update_population(population)

region.load_migration_matrix(['RECREATION'], 
                            matrix_dir='data/provision',
                            use_updated=use_updated, # Внимание! Если True, использовать новое население. Если False, использовать базовое.
                            average=True,
                            anchors=anchor_settlement)

In [ ]:
mobility = region.analyze_mobility(anchor_threshold=90)

# 5. Генерация карты миграции

In [ ]:
fmap = region.create_map()
fmap